In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import soundfile as sf
import librosa
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from IPython.display import clear_output
import matplotlib

df = pd.read_excel('ins.xlsx')
df.rename(columns={"Unnamed: 3": "beat_counter", "Unnamed: 4": "defect_counter"}, inplace=True)
df.beat_counter = df.beat_counter.fillna('1')
df.defect_counter = df.defect_counter.fillna('1')

df.loc[(df.defect_counter =='?')|(df.defect_counter =='!'), 'defect_counter'] = '1' 

df.loc[123, 'end'] = 620320
df.loc[126, 'end'] = 3590000

ind = df.loc[df.file == '00004-2022-08-02-07-43-27_channel_3.bin'].index[-1]
df.loc[ind, 'file'] = '00004-2022-08-02-07-43-27_channel_10.bin'

df.file = df.file.apply(lambda x: x[6:])
df.loc[df.file == '2022-08-02-07-43-27_channel_3.bin', 'file'] = '2022-08-02-07-43-27_channel_03.bin'

print(df.defect_counter.unique())


ModuleNotFoundError: No module named 'torch'

In [ ]:
def construct_examples(audio_path, win_len=2.56, hop_len=1.0, sr=112000):
    """
    Constructs audio examples of window length and step (hop_len).
    
    Parameters:
        audio_path (str): Path to the audio file.
        win_len (float): Window length in seconds.
        hop_len (float): Hop length in seconds.
        sr (int): Sample rate of the audio.
    
    Returns:
        Tuple: A tuple containing an array of audio examples and corresponding window ranges.
    """

    # Read audio file
    audio = np.fromfile(audio_path, dtype='uint8')

    # Convert window and hop lengths from seconds to samples
    win_len_samples = int(sr * win_len)
    hop_len_samples = int(sr * hop_len)

    # Calculate the number of hops
    no_of_hops = math.ceil((len(audio) - win_len_samples) / hop_len_samples)

    # Pad audio to fit exact number of windows
    audio_padded = np.pad(audio, (0, win_len_samples + hop_len_samples * no_of_hops - len(audio)), mode='constant')

    # Generate examples
    a_ex = [audio_padded[i: i + win_len_samples] for i in range(0, len(audio_padded) - win_len_samples + 1, hop_len_samples)]

    # Calculate window ranges in seconds
    win_ranges = [(i / sr, (i + win_len_samples) / sr) for i in range(0, len(audio_padded) - win_len_samples + 1, hop_len_samples)]

    return a_ex, win_ranges


In [ ]:
def construct_labels(annotation_df, win_start, win_end, win_len, sr=112000):
    """
    Calculates all events in the specified window.
    
    Parameters:
        annotation_df (pd.DataFrame): DataFrame containing audio annotations.
        win_start (float): Start of the window in seconds.
        win_end (float): End of the window in seconds.
        win_len (float): Length of the window in seconds.
        sr (int): Sample rate of the audio.
    
    Returns:
        list: A list of events, each represented as [start, end, class] in seconds.
    """

    # Convert start and end times in the DataFrame from samples to seconds
    start_times_sec = annotation_df['start'] / sr
    end_times_sec = annotation_df['end'] / sr

    # Initialize an empty list for events that occur in the specified window
    events_in_window = []

    # Iterate over each annotation
    for start, end, event_class in zip(start_times_sec, end_times_sec, annotation_df['defect_counter']):
        # Check if the event is within the specified window
        if start < win_end and end > win_start:
            # Adjust the start and end times of the event relative to the window
            adjusted_start = max(start, win_start) - win_start
            adjusted_end = min(end, win_end) - win_start

            # Append the adjusted event to the list
            events_in_window.append([adjusted_start, adjusted_end, event_class])

    # If no events are found, append a 'silent' event spanning the whole window
    if len(events_in_window) == 0:
        events_in_window.append([0, win_len, '0'])

    return events_in_window


In [ ]:
def get_universal_labels(events, class_dict, ex_length=10.0, no_of_div=32):
    """
    Generates labels for each divided part of the window.
    
    Parameters:
        events (list): List of events in the window.
        class_dict (dict): Dictionary of classes.
        ex_length (float): Length of the incoming window.
        no_of_div (int): Number of divisions in the window.
    
    Returns:
        np.array: Array of labels for each part of the window.
    """

    # Calculate the length of each segment
    segment_length = ex_length / no_of_div

    # Initialize the label array
    labels = np.zeros((no_of_div, len(class_dict) * 3))

    # Process each event
    for event in events:
        event_start, event_end, event_class = event
        event_class_idx = class_dict[event_class] * 3

        # Determine the start and end segments for the event
        start_segment = int(event_start // segment_length)
        end_segment = int(event_end // segment_length)

        # Label the segments
        for seg in range(start_segment, min(end_segment + 1, no_of_div)):
            segment_start = seg * segment_length
            segment_end = segment_start + segment_length
            labels[seg, event_class_idx] = 1  # Mark the presence of the class
            labels[seg, event_class_idx + 1] = max(event_start - segment_start, 0) / segment_length  # Start position in segment
            labels[seg, event_class_idx + 2] = min(event_end - segment_start, segment_length) / segment_length  # End position in segment

    return labels


In [ ]:
# Extracting unique file names from the dataframe
all_files = df.file.unique()

# Defining the base folder path where the files are located
folder_path = 'Marked data/'

# A common prefix to be added to each file name
prefix = '00004-'

# Constructing the full file paths
all_files = [folder_path + prefix + file[:-15] + '/' + prefix + file for file in all_files]


In [ ]:
df1 = pd.read_excel('Marked data/part2/ins.xlsx')

In [ ]:
all_files = df1.file.unique()
print(all_files)
folder_path = 'Marked data/part2/'
#prefix = '00004-'
all_files = [folder_path+file for file in all_files]
all_files, len(all_files), len('Marked data/00004-')

In [ ]:
df1.loc[66, ['start', 'end']] = [503484, 755824]

In [ ]:
df1.rename(columns={"Сколько импульсов": "beat_counter", "Сколько дефектов": "defect_counter"}, inplace=True)

In [ ]:
df1.defect_counter = df1.defect_counter.astype('str')

In [ ]:
for i, audio in enumerate(all_files):
    annotation_df = df1.loc[df1.file == audio[len(folder_path):]]
    
    a, win_ranges = construct_examples(audio, win_len=win_len, hop_len=hop_len) # возвращаем массив окон для ффт и отрезки в секундах
    a_new_ex_train += a   # добавляем отрезки в массив
    
    for w in win_ranges:
        # для каждого отрезка пишем события, которые попали в отрезок
        labels_t = construct_labels(annotation_df, w[0], w[1], win_len=win_len)
        ll = get_universal_labels(labels_t, binary_class_dict, ex_length=win_len, no_of_div = 9)
        # потом отрезок делим на бины и записываем какие события попали в бины
        # ll = to_seg_by_class(labels_t, class_dict)
        a_new_labels_train.append(ll)

In [ ]:
## Create custom data

In [ ]:
not_null_indexes = []
for i, label in enumerate(a_new_labels_train):
    if np.sum(label[:, 3:]) != 0:
        not_null_indexes.append(i)
    
len(not_null_indexes)

In [ ]:
train_samples = np.array(a_new_ex_train)[not_null_indexes]
train_labels  = np.array(a_new_labels_train)[not_null_indexes]

In [ ]:
import librosa
sr = 112000
def get_log_melspectrogram(audio, sr = sr, hop_length = 1120, win_length = 4480, n_fft = 4480, n_mels = 70, fmin = 0, fmax = 56000):
    """Return the log-scaled Mel bands of an audio signal."""
    audio_2 = librosa.util.normalize(audio)
    bands = librosa.feature.melspectrogram(
        y=audio_2, sr=sr, hop_length=hop_length, win_length = win_length, n_fft=n_fft, n_mels=n_mels)
    return librosa.core.power_to_db(bands)

In [ ]:
for i, a in enumerate(train_samples):
    M = get_log_melspectrogram(a).T
    np.save("custom_data/train_data/ex-" + str(i) + ".npy", M)

In [ ]:
for i, a in enumerate(train_labels):
    np.save("custom_data/train_data/label-" + str(i) + ".npy", a)

In [ ]:
import re

def tryint(s):
    try:
        return int(s)
    except ValueError:
        return s
    
def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)

In [ ]:
import glob
import random
"""
Load the individual numpy arrays into partition
"""
data = glob.glob("custom_data/train_data/ex-*.npy")  
sort_nicely(data)

labels = glob.glob("custom_data/train_data/label-*.npy") 
sort_nicely(labels)

train_examples = [(data[i], labels[i]) for i in range(len(data))]

random.seed(4)
random.shuffle(train_examples)

In [ ]:
class SedDataset(Dataset):
    
    def __init__(self, train_examples, sr=sr):
        self.train_examples = train_examples
        self.sr = sr
        
    def __len__(self):
        return len(self.train_examples)
    
    def __getitem__(self, idx):
        
        X = np.load(self.train_examples[idx][0])
        y = np.load(self.train_examples[idx][1])
        
        return torch.from_numpy(X), torch.from_numpy(y)

In [ ]:
dataset = SedDataset(train_examples)

In [ ]:
dataloader = DataLoader(dataset, batch_size=32,
                        shuffle=True, num_workers=0)

In [ ]:
for i, (X1, y1) in enumerate(dataloader):
    print(X1.shape, y1.shape)
    print(torch.sum(y1[:,:,3:]))
    if i>6:
        break

In [ ]:
### Create NN

In [ ]:
def binary_yoho_loss(y_true, y_pred):
    """
    my custom loss for yoho task
    """
    
    squared_difference = torch.square(y_true - y_pred) #[batch, 9, 12]
    #print(squared_difference.shape)
    ss_True = squared_difference[:, :, 0] * 0 + 1 #[batch, 9] of ones
    ss_0 = y_true[:, :, 0] #[batch, 9]
    ss_1 = y_true[:, :, 3]
    
    sss = torch.stack((ss_True, ss_0, ss_0,
                  ss_True, ss_1, ss_1), axis = 2)
    
    squared_difference =  torch.multiply(squared_difference, sss)
    return torch.mean(squared_difference)

In [ ]:
class DepthwiseSeparableConv2D(nn.Module):
    def __init__(self, nin, nout, kernel_size=3, stride = 1, padding=0):
        super(DepthwiseSeparableConv2D, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=kernel_size, stride=stride,padding=padding)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [ ]:
class Reshape(nn.Module):
    def __init__(self, args):
        super(Reshape, self).__init__()
        self.my_shape = args

    def forward(self, x):
        return x.view(*self.my_shape)

In [ ]:
model = nn.Sequential(
            nn.Conv2d(1, 32, 3, 2),
            nn.ReLU(),
#            nn.Conv2d(32, 32, kernel_size=3),
#            nn.Conv2d(32, 64, kernel_size=1),
            DepthwiseSeparableConv2D(32, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
#            nn.Conv2d(64, 64, kernel_size=3, stride = 2),
#            nn.Conv2d(64, 128, kernel_size=1),
            DepthwiseSeparableConv2D(64, 128, 3, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
#            nn.Conv2d(128, 128, kernel_size=3),
#            nn.Conv2d(128, 256, kernel_size=1),
            DepthwiseSeparableConv2D(128, 256, 3),
            nn.BatchNorm2d(256),
            nn.ReLU(),
#            nn.Conv2d(256, 256, kernel_size=3, stride = 2),
#            nn.Conv2d(256, 512, kernel_size=1),
            DepthwiseSeparableConv2D(256, 512, 3, stride=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
#            nn.Conv2d(512, 512, kernel_size=3),
#            nn.Conv2d(512, 1024, kernel_size=1),
            DepthwiseSeparableConv2D(512, 1024, 3),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
#            nn.Conv2d(1024, 1024, kernel_size=3, padding = 1),
#           nn.Conv2d(1024, 512, kernel_size=1),
            DepthwiseSeparableConv2D(1024, 512, 3, padding = 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
#            nn.Conv2d(512, 512, kernel_size=3, padding = 1),
#            nn.Conv2d(512, 256, kernel_size=1),
            DepthwiseSeparableConv2D(512, 256, 3, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
#            nn.Conv2d(256, 256, kernel_size=3, padding = 1),
#            nn.Conv2d(256, 128, kernel_size=1),
            DepthwiseSeparableConv2D(256, 128, 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            Reshape((-1, 512, 27)), 
            nn.Conv1d(512, 128, kernel_size=7),
            nn.ReLU(),
            nn.Conv1d(128, 64, kernel_size=7),
            nn.ReLU(),
            nn.Conv1d(64, 32, kernel_size=4),
            nn.ReLU(),
            nn.Conv1d(32, 16, kernel_size=4),
            nn.ReLU(),
            nn.Conv1d(16, 9, kernel_size=4)
            )

In [ ]:
for x, y in dataloader:
    print(x.shape)
    i = x.reshape([-1, 1, 257, 70]).float()
    i = model(i)
    print(i.shape)
    break

In [ ]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_files, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_files, batch_size=batch_size, shuffle=False)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    model.to(device)
    training_loss = []
    val_loss = [] 
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0

        for i, (X, y) in enumerate(train_loader, 0):
            model.train()

            # get the inputs
            images = X.to(device).float()
            labels = y.to(device).float()

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict classes using images from the training set
            outputs = model(images.reshape([-1, 1, 257, 70]))
            # compute the loss based on model output and real labels
            loss = binary_yoho_loss(labels, outputs)
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()
            
            running_loss += loss.cpu().detach().numpy()/batch_size
            
        training_loss.append(running_loss)    # extract the loss value

 
        val_running = 0.0
        for i, (X, y) in enumerate(val_loader, 0):
            model.eval()

            val_images = X.to(device).float()
            val_labels = y.to(device).float()

            out = model(val_images.reshape([-1, 1, 257, 70]))
            
            val_running += binary_yoho_loss(val_labels, out).cpu().detach().numpy()/batch_size
            
        val_loss.append(val_running)
        
       # clear_output(wait=True)
       # plt.plot(running_loss, label='train_loss')
       # plt.plot(val_loss,label='val_loss')
       # plt.legend()
       # plt.show
    
    return training_loss, val_loss

In [ ]:
device = torch.device("cuda")
device

In [ ]:
train_files, val_files = train_test_split(dataset, test_size=0.15)
train_loss, val_loss = train(train_files, val_files, model, 60, 32)
plt.plot(train_loss[5:], label='train_loss')
plt.plot(val_loss[5:],label='val_loss')
plt.legend()
plt.show